# **핵심만 요약한 통계와 머신러닝 파이썬 코드북 개정1판**
- ⓒ2023 AlgoBoni all rights reserved.
- 본 컨텐츠의 저작권은 알고보니에 있습니다. 저작권법에 의해 보호를 받는 저작물이므로 무단 전재와 무단 복제를 금합니다.
- 본 컨텐츠의 종이책은 [교보문고](https://product.kyobobook.co.kr/detail/S000209591909), [예스24](https://www.yes24.com/Product/Goods/122661688), [알라딘](https://www.aladin.co.kr/shop/wproduct.aspx?ISBN=K262935029&start=pnaver_02)에서 구매할 수 있습니다. 종이책에서는 아래 개념 및 코드에 대한 설명과 연습문제를 제공합니다.

# 7. 공분산과 상관계수
## 7-1. 공분산 

In [ ]:
print("[수기 계산]")
import numpy as np
X = np.array([4,6,6,8,8,9,9,10,12,12])
Y = np.array([39,42,45,47,50,50,52,55,57,60])
Xm = np.mean(X)
Ym = np.mean(Y)
n = len(X)

# 공분산을 계산하는 두 가지 방법
cov1 = (np.sum(X*Y) - n*Xm*Ym) / (n-1)
cov2 = sum((X - Xm)*(Y - Ym)) / (n-1)
print(" {:.2f}, {:.2f}".format(cov1, cov2))

[수기 계산]
 16.80, 16.80


In [ ]:
print("[라이브러리 계산]")
cov3 = np.cov(X, Y) # numpy는 공분산 행렬을 반환해줌
print(DataFrame(cov3, columns=['X', 'Y'], index=['X', 'Y']))
print(" X-Y의 공분산: {:.2f}".format(cov3[0][1]))

[라이브러리 계산]
           X          Y
X   6.711111  16.800000
Y  16.800000  44.011111
 X-Y의 공분산: 16.80


## 7-2. 상관계수

### - 피어슨 상관계수

In [124]:
# 피어슨 상관계수 (=표본상관계수)
print("[수기 계산]")
X = np.array([4,6,6,8,8,9,9,10,12,12])
Y = np.array([39,42,45,47,50,50,52,55,57,60])

# 표본상관계수 계산하는 두 가지 방법
Xm = np.mean(X)
Ym = np.mean(Y)
n = len(X)
corr1 = (np.sum(X*Y) - n*Xm*Ym) / np.sqrt(sum((X-Xm)**2) * sum((Y-Ym)**2))

Sx = np.sqrt(sum((X - Xm)**2) / (n-1)) # X표본표준편차
Sy = np.sqrt(sum((Y - Ym)**2) / (n-1)) # Y표본표준편차
cov1 = (np.sum(X*Y) - n*Xm*Ym) / (n-1) #X, Y의 공분산
corr2 = cov1 / (Sx*Sy) # 공분산을 X, Y의 표본표준편차들로 나누어서 구한다.
print(" {:.2f}, {:.2f}".format(corr1, corr2))

[수기 계산]
 0.98, 0.98


In [125]:
print("[라이브러리 계산]")
from scipy.stats import pearsonr
corr3 = np.corrcoef(X,Y)[0][1]
corr4, pval = pearsonr(X, Y) # 피어슨 상관계수와 유의확률을 반환해줌
print(" {:.2f}, {:.2f} (p-value: {:.2f})".format(corr3, corr4, pval))

[라이브러리 계산]
 0.98, 0.98 (p-value: 0.00)


In [126]:
# 모상관계수의 점추정량과 신뢰구간 계산: Z분포 사용
conf_a = 0.05
conf_z = norm.ppf(1-conf_a/2) #or 1.96
r1 = corr1 #위에서 계산한 표본상관계수 corr1
n = len(X) #표본의 개수
a = 1/2 * np.log((1+r1)/(1-r1)) -conf_z/np.sqrt(n-3)
b = 1/2 * np.log((1+r1)/(1-r1)) +conf_z/np.sqrt(n-3)
conf1 = (np.exp(2*a)-1) / (np.exp(2*a)+1) # = np.tanh(a)
conf2 = (np.exp(2*b)-1) / (np.exp(2*b)+1) # = np.tanh(b)

print("[추정]")
print(" 점 추정량: {:.3f}".format(r1))
print(" 구간 추정량: {:.3f}~{:.3f}".format(conf1, conf2))

[추정]
 점 추정량: 0.978
 구간 추정량: 0.905~0.995


In [127]:
# 모상관계수의 가설 검정: t분포 사용, 표본이 충분히 큰 경우에는 Z분포 사용
# H0: 모상관계수는 0이다, H1: 모상관계수는 0이 아니다. (통계적으로 유의하다)
from scipy.stats import t, norm
r0 = 0 #귀무가설의 모상관계수
r1 = corr1 #위에서 계산한 표본상관계수 corr1
n = len(X) #표본의 개수
df = n-2 #자유도
tstat = np.sqrt(df) * (r1-r0) / np.sqrt(1-r1**2) #검정통계량 t
#zstat = 1/2 * np.log((1+r1)/(1-r1)) # 검정통계량 Z
test_a = 0.05
sp = (1-t.cdf(np.abs(tstat), df))*2 #양측검정
cv = t.ppf(1-test_a/2, df)
cv = "+/- {:.3f}".format(cv)

print("[검정]") # 귀무가설을 기각할 수 없음!
print(" 임계값: {}, 검정통계량: {:.3f}".format(cv, tstat))
print(" 유의수준: {:.3f}, 유의확률: {:.3f}".format(test_a, sp))

[검정]
 임계값: +/- 2.306, 검정통계량: 13.117
 유의수준: 0.050, 유의확률: 0.000


In [ ]:
print("[라이브러리 검정]")
from scipy.stats import pearsonr
corr, p = pearsonr(X, Y)
print(" 상관계수: {:.3f}, p-value: {:.3f}".format(corr, p))
## 검정 결과, 대립가설을 채택하여 모상관계수가 0이 아니기 때문에 구해진 상관계수는 통계적으로 유의하다는 결론을 얻었다.

[라이브러리 검정]
 상관계수: 0.978, p-value: 0.000


### - 스피어만 순위상관계수

In [ ]:
## 표본 내 같은 순위가 없는 경우
X = np.array([1,2,3,4,5,6,7,8,9,10])
Y = np.array([4,3,5,2,6,1,10,7,8,9])

print("[수기 계산]")
rtable = DataFrame({"X":X, 'Y':Y})
rtable['Xr'] = rtable['X'].rank(ascending=False)
rtable['Yr'] = rtable['Y'].rank(ascending=False)
corr1 = 1 - 6 * sum((rtable['Xr']-rtable['Yr'])**2)/len(X)/(len(X)**2-1)
print(" 표본 내 동일한 데이터가 없는 경우 {:.3f}".format(corr1))

[수기 계산]
 표본 내 동일한 데이터가 없는 경우 0.661


In [ ]:
print("[라이브러리 계산]")
from scipy.stats import spearmanr
corr2, p = spearmanr(X, Y)
print(" 표본 내 동일한 데이터가 없는 경우 {:.3f}, p-value: {:.3f}".format(corr2, p))
## p-value가 유의수준 0.05보다 작으므로 해당 상관계수는 통계적으로 유의!

[라이브러리 계산]
 표본 내 동일한 데이터가 없는 경우 0.661, p-value: 0.038


In [ ]:
## 표본 내 같은 순위가 있는 경우
X2 = np.array([10,9,9,8,8,8])
Y2 = np.array([6,7,8,8,7,6.5])

print("[수기 계산]")
rtable = DataFrame({"X2":X2, 'Y2':Y2})
rtable['X2r'] = rtable['X2'].rank(ascending=False)
rtable['Y2r'] = rtable['Y2'].rank(ascending=False)

#C: 같은 순위에 포함된 데이터 개수
#Clist: C들이 포함된 리스트
Cxlist = DataFrame(rtable['X2r'].value_counts()).query("X2r >= 2").values.ravel()
Cylist = DataFrame(rtable['Y2r'].value_counts()).query("Y2r >= 2").values.ravel()

#Cx와 Cy 구하기
Cx = 0
Cy = 0
for cx, cy in zip(Cxlist, Cylist):
    Cx += cx*(cx**2-1)
    Cy += cy*(cy**2-1)

#Tx와 Ty 구하기
Tx = 1/12 * (len(X2)*(len(X2)**2-1) - Cx)
Ty = 1/12 * (len(Y2)*(len(Y2)**2-1) - Cy)

corr3 = (Tx+Ty-sum((rtable['X2r']-rtable['Y2r'])**2)) / (2* np.sqrt(Tx*Ty))
print(" 표본 내 동일한 데이터가 있는 경우 {:.3f}".format(corr3))

[수기 계산]
 표본 내 동일한 데이터가 있는 경우 -0.318


In [ ]:
print("[라이브러리 계산]")
from scipy.stats import spearmanr
corr4, p = spearmanr(X2, Y2)
print(" 표본 내 동일한 데이터가 있는 경우 {:.3f}, p-value: {:.3f}".format(corr4, p))
## 유의수준 0.05 기준으로 해당 상관계수는 통계적으로 유의하다고 보기 어렵다.

[라이브러리 계산]
 표본 내 동일한 데이터가 있는 경우 -0.318, p-value: 0.539


### - 켄달의 타우

In [ ]:
## 표본 내 같은 순위가 없는 경우
def combi(x, y):
    from math import factorial as fact
    return fact(x)/fact(x-y)/fact(y)

X = np.array([80,70,60,50,95,85,77,55,65])
Y = np.array([6,3,2,1,8,9,7,4,5])

print("[수기 계산]")
One = 0
mOne = 0
for i in range(0, len(X)):
    for j in range(i, len(X)):
        if (X[i]-X[j])*(Y[i]-Y[j]) > 0:
            One += 1
        elif (X[i]-X[j])*(Y[i]-Y[j]) < 0:
            mOne += 1

corr1 = (1*One -1*mOne) / combi(len(X), 2)
print(" 표본 내 동일한 데이터가 없는 경우 {:.3f}".format(corr1))

[수기 계산]
 표본 내 동일한 데이터가 없는 경우 0.722


In [ ]:
from scipy.stats import kendalltau
print("[라이브러리 계산]")
corr2, p = kendalltau(X, Y)
print(" 표본 내 동일한 데이터가 없는 경우 {:.3f}, p-value: {:.3f}".format(corr2, p))

[라이브러리 계산]
 표본 내 동일한 데이터가 없는 경우 0.722, p-value: 0.006


In [ ]:
## 표본 내 같은 순위가 있는 경우
def combi(x, y):
    from math import factorial as fact
    return fact(x)/fact(x-y)/fact(y)

X2 = np.array([10,9,9,8,8,8])
Y2 = np.array([6,7,8,8,7,6.5])

print("[수기 계산]")
One = 0
mOne = 0
for i in range(0, len(X2)):
    for j in range(i, len(X2)):
        if (X2[i]-X2[j])*(Y2[i]-Y2[j]) > 0:
            One += 1
        elif (X2[i]-X2[j])*(Y2[i]-Y2[j]) < 0:
            mOne += 1
            
#Cx, Cy:같은 순위에 포함된 데이터 개수
rtable = DataFrame({"X2":X2, 'Y2':Y2})
rtable['X2r'] = rtable['X2'].rank(ascending=False)
rtable['Y2r'] = rtable['Y2'].rank(ascending=False)

#Clist: C들이 포함된 리스트
Cxlist = DataFrame(rtable['X2r'].value_counts()).query("X2r>=2").values.ravel()
Cylist = DataFrame(rtable['Y2r'].value_counts()).query("Y2r>=2").values.ravel()

#Cx와 Cy 구하기
Cx = 0
Cy = 0
for cx, cy in zip(Cxlist, Cylist):
    Cx += 1/2*cx*(cx-1)
    Cy += 1/2*cy*(cy-1)

#Tx와 Ty 구하기
Tx = combi(len(X2), 2) - Cx
Ty = combi(len(X2), 2) - Cy
    
corr3 = (1*One -1*mOne) / np.sqrt(Tx*Ty)
print(" 표본 내 동일한 데이터가 있는 경우 {:.3f}".format(corr3))

[수기 계산]
 표본 내 동일한 데이터가 있는 경우 -0.251


In [ ]:
print("[라이브러리 계산]")
from scipy.stats import kendalltau
corr4, p = kendalltau(X2, Y2)
print(" 표본 내 동일한 데이터가 있는 경우 {:.3f}, p-value: {:.3f}".format(corr4, p))

[라이브러리 계산]
 표본 내 동일한 데이터가 있는 경우 -0.251, p-value: 0.524


### - 크라메르의 연관계수

In [ ]:
print("[수기 계산]")
from pandas import DataFrame
import numpy as np
table = DataFrame({"성별":['남자', '여자'], '안경O':[10,30], '안경X':[40, 20]}).set_index('성별')
print("데이터 확인:\n", table)

[수기 계산]
데이터 확인:
     안경O  안경X
성별          
남자   10   40
여자   30   20


In [ ]:
n = table.sum().sum() #전체 데이터 개수
exp = []
r = table.sum(axis=1).values
c = table.sum(axis=0).values

for R in r:
    for C in c:
        exp.append(R*C/n)
        
obs = table.values.ravel()
chistat = np.sum((obs - exp)**2 / exp)

k, l = table.shape[0], table.shape[1] #각 변수의 범주의 개수
V = np.sqrt(chistat/(np.minimum(k, l)-1)/n)

print(" 연관계수: {:.3f}".format(V))

 연관계수: 0.408


In [ ]:
print("[라이브러리 계산]")
from scipy.stats.contingency import association
V2 = association(table.values, method='cramer')
print(" 연관계수: {:.3f}".format(V2))

[라이브러리 계산]
 연관계수: 0.408



# 연습문제

### - 1번 문제 풀이

In [129]:
from scipy.stats import pearsonr, spearmanr
math = [96, 93, 63, 89, 85, 84, 66, 62, 90]
eng = [98, 90, 74, 84, 69, 69, 73, 61, 98]
pr, p1 = pearsonr(math, eng)
sr, p2 = spearmanr(math, eng)
print("피어슨 상관계수 {:.2f} (p-value {:.2f})".format(pr, p1))
print("스피어만 상관계수 {:.2f} (p-value {:.2f})".format(sr, p2))
# 두 상관계수 모두 유의확률이 유의수준 0.05보다 작기 때문에 통계적으로 유의하다.

피어슨 상관계수 0.75 (p-value 0.02)
스피어만 상관계수 0.82 (p-value 0.01)


In [130]:
# 피어슨 상관계수의 신뢰구간 계산
from scipy.stats import norm
import numpy as np
conf_a = 0.1 # 신뢰수준 90%
conf_z = norm.ppf(1-conf_a/2)

r1 = pr #위에서 계산한 표본상관계수 pr을 사용
n = 9 #표본의 개수
a = 1/2 * np.log((1+r1)/(1-r1)) -conf_z/np.sqrt(n-3)
b = 1/2 * np.log((1+r1)/(1-r1)) +conf_z/np.sqrt(n-3)
conf1 = (np.exp(2*a)-1) / (np.exp(2*a)+1)
conf2 = (np.exp(2*b)-1) / (np.exp(2*b)+1)
print(" 구간 추정량: {:.3f}~{:.3f}".format(conf1, conf2))

 구간 추정량: 0.283~0.927


### - 2번 문제 풀이

In [121]:
xs, ys = 4, 5
cov = 4
cov/(xs*ys)

0.2